# Transformée en Ondelettes 2D, application au traitement des images

In [ ]:
import numpy as np
import scipy as scp
import pylab as pyl
import matplotlib.pyplot as plt
import pywt
import scipy.io as sio
import pandas as pd
import holoviews as hv
import param
import panel as pn
from panel.pane import LaTeX
hv.extension('bokeh')
from PIL import Image
from io import BytesIO
import requests
import boto3
import os

In [ ]:
s3_endpoint_url = 'https://object-rook-ceph.apps.math.cnrs.fr/'
s3_access_key_id = '9F7EB8YBUWXDV7A4IZYW' # le contenu de secrets/dossal
s3_secret_access_key = 'skV01Eei5M3xVOxROIDr3qymYhWtkrxPpMyj8nwb' # le contenu de secrets/dossal
s3_bucket = 'signal-image'
s3 = boto3.client('s3',
                  '',
                  endpoint_url = s3_endpoint_url,
                  aws_access_key_id = s3_access_key_id,
                  aws_secret_access_key = s3_secret_access_key)
Data=["Lenna.jpg","Canaletto.jpeg","MinotaureBruite.jpeg","Cartoon.jpg"]
if not os.path.isfile('Lenna.jpg'):
    for filenames in Data:  
        s3.download_file(s3_bucket,filenames,filenames)
def chargeData(name):
    if name=='Lenna':
        res=np.array(Image.open("Lenna.jpg")).astype(float)
    if name=='Canaletto':
        res=np.array(Image.open("Canaletto.jpeg")).astype(float)
    if name=='Minotaure':
        res=np.array(Image.open("MinotaureBruite.jpeg")).astype(float)  
    if name=='Cartoon':
        res=np.array(Image.open("Cartoon.jpg")).astype(float) 
    return res
options1=dict(width=400,height=400,xaxis=None,yaxis=None,toolbar=None)
options2=dict(width=700,height=400,xaxis=None,yaxis=None,toolbar=None)

## Approximation linéaire et non linéaire.

In [ ]:
im2=chargeData('Lenna')
im=chargeData('Canaletto')
imagesRef= {"Lenna" : im2,"Canaletto" : im}
options = dict(cmap='gray',xaxis=None,yaxis=None,width=400,height=400,toolbar=None)
pn.Row(hv.Raster(imagesRef["Lenna"]).opts(**options),hv.Raster(imagesRef["Canaletto"]).opts(**options))

In [ ]:
size=400
WT= pywt.wavedecn(im, 'haar', mode='per', level=2)
arr, coeff_slices = pywt.coeffs_to_array(WT)
hv.Image(arr).opts(cmap='gray',width=size,height=size)

Ecrire une fonction qui réalise une approxiamtion non linéaire en seuillant les coefficients d'ondelettes.
On pourra utiliser les fonctions suivante : pywt.coeffs_to_array et pywt.array_to_coeffs

In [ ]:
def ApproxOnd2D(S,qmf,L,threshold):
    
    # On décompose l'image S avec la base d'ondelettes qmf
    WTB= pywt.wavedecn(S, qmf, mode='per', level=L)
    arr, coeff_slices = pywt.coeffs_to_array(WTB)
    
    # On applique le seuillage doux : on met à 0 les coefficients inférieur au seuil
    WTS=arr*(np.abs(arr)>threshold)
    
    # On reconstruit l'image avec la base d'ondelettes et les nouveaux coefficients
    ncoeffs = pywt.array_to_coeffs(WTS, coeff_slices)
    Srec=pywt.waverecn(ncoeffs,qmf,mode='per')
    
    return Srec,ncoeffs

Ecrire une focntion PSNR.

In [ ]:
def PSNR(I,Iref):
    
    # On calcule l'erreur quadratique moyenne
    mse = np.mean( (Iref - I) ** 2 )
    
    # Si l'erreur entre les deux images est nulle. cela signifie que l'image est parfaitement reconstruite donc le PSNR = 100
    if mse == 0:
        return 100
    
    Val_MAX = np.max(Iref)
    return 20 * np.log10(Val_MAX / np.sqrt(mse))

Ecrire une fonction qui réalise une approximation non linéaire en conservant un nombre N de coefficients d'ondelettes et la tester. On pourra utiliser les fonctions pywt.ravel_coeffs et unravel_coeffs.

In [ ]:
def ApproxOnd2nonlin(I,qmf,L,N):
    
    rows, cols = I.shape
    N1 = rows*cols
    
    # On cherche le niveau maximum possible de décomposition dans cette base
    Lmax = pywt.dwtn_max_level((rows,cols), pywt.Wavelet(qmf))
    L1 = int(min(L, Lmax))

    WT = pywt.wavedec2(I, qmf, mode='per', level=L1)
    arr, coeff_slices, shapes = pywt.ravel_coeffs(WT)

    # On garde les N plus grand coefficients 
    # C'est similaire à précédemment sauf que cette fois-ci on ne choisit pas un seuil mais un nombre de coefficients à garder
    # D'où le fait qu'on prenn les plus importants.
    Ind = np.argsort(np.abs(arr))
    WTS = np.zeros(N1)
    WTS[Ind[int(N1-N):int(N1)]] = arr[Ind[int(N1-N):int(N1)]]

    coeffs_from_arr = pywt.unravel_coeffs(WTS, coeff_slices,shapes)
    Irec = pywt.waverecn(coeffs_from_arr, qmf, mode='per')
    
    p = PSNR(I, Irec)
    return Irec, p

Créer un Dashboard qui permet d'explorer la fonction précédente.

In [ ]:
wavelist = ['haar','db2','db3','db4','coif1','coif2','coif3']

In [ ]:
class Approx2D(param.Parameterized):
    image = param.ObjectSelector(default="Canaletto",objects=imagesRef.keys())
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    L = param.Integer(5,bounds=(0,10))
    N = param.Integer(2000,bounds=(1,10000))
  #  @param.depends('wave', 'N', 'L')
    def view(self):
        Irec, p = ApproxOnd2nonlin(imagesRef[self.image],self.wave,self.L,self.N)
        return pn.Column(hv.Image(Irec).opts(cmap='gray',width=400,height=400))

In [ ]:
approx2D = Approx2D()
#pn.Row(approx2D.param,approx2D.view)

Créer un plan d'experiences qui permet d'explorer la fonction ApproxOnd2nonlin

In [ ]:
import itertools
wavelist = ['haar','db2','db3','db4','coif1','coif2','coif3']
experiences = {'Image':imagesRef,'N':np.linspace(1000,50000,30),'wave':wavelist,'L':np.linspace(1,10,5)}
dfexp = pd.DataFrame(list(itertools.product(*experiences.values())),columns=experiences.keys())

Créer la fonction qui à une ligne de la base de donnée précédente calcule le PSNR associé.

In [ ]:
def row2PSNR(row):
    I,p=ApproxOnd2nonlin(imagesRef[row.Image], row['wave'],  row['L'], row['N'])
    return {'PSNR':p}

Appliquer la fonction sur la base de donnée et ajouter la colonne PSNR à la base de données dfexp 

In [ ]:
result2 = dfexp.apply(row2PSNR, axis=1)
print(result2)

In [ ]:
dfexp['PSNR'] = pd.DataFrame.from_records(result2)['PSNR']

In [ ]:
print(dfexp)

Utiliser hvplot pour visualiser la base de données.

In [ ]:
import hvplot.pandas

In [ ]:
from bokeh.models import HoverTool
h = HoverTool()
dfexp.hvplot('N','PSNR',by='wave',kind='scatter',groupby=['Image','L'])\
.opts(width=600,tools = [h]).redim.range(PSNR=(20,70),translation=(-0.5,10.5))

On peut remarquer que pour les images, plus L augmente, plus le PSNR en fonction de N est linéaire. De plus, on observe que le PSNR avec la base Haar est le plus faible, et celui avec la base coif3 est meilleur.

## Débruitage d'images

Ecrire une fonction qui effectue un seuillage dur en ondelettes et la tester. On pourra utiliser la fonction pywt.ravel_coeffs et on pensera à cliper le résultat entre 0 et 255.

In [ ]:
def SeuillageDurOndelettes(I,qmf,L,Seuil):
    
        WTB= pywt.wavedecn(I, qmf, mode='per', level=L)
        coeff_arr, coeff_slices, coeff_shapes = pywt.ravel_coeffs(WTB)
        WTS=coeff_arr*(np.abs(coeff_arr)>Seuil)
        coeffs_from_arr = pywt.unravel_coeffs(WTS, coeff_slices,coeff_shapes)
        Irec=pywt.waverecn(coeffs_from_arr,qmf,mode='per')
        
        # On clip l'image pour avoir des valeurs d'intensité de gris entre 0 et 255.
        Irec=np.clip(Irec,0,255)
        
        return Irec

Construire un dashboard qui permet d'explorer la fonction SeuillageDurOndelettes.

In [ ]:
class WaveSeuillage(param.Parameterized):
    image = param.ObjectSelector(default="Canaletto",objects=imagesRef.keys())
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    L = param.Integer(7,bounds=(0,7))
    Seuil = param.Number(10,bounds=(1,1000))
    def view(self):
        Irec = SeuillageDurOndelettes(imagesRef[self.image],self.wave,self.L,self.Seuil)
        return pn.Column(hv.Image(Irec).opts(cmap='gray',width=400,height=400))

In [ ]:
waveseuil = WaveSeuillage()
pn.Row(waveseuil.param,waveseuil.view)

On observe que plus le seuil augmente, plus l'image noircie. En effet dans notre code, on ne garde que les coefficients qui sont supérieurs au seuil. Si le pixel apparait noir c'est que le coefficient correspondant a été mis à nul.
On remarque également que plus L est grand, plus on perd de l'information. C'est du au fait que l'on décompose notre image en petits carrés de taille L*L.

In [ ]:
n1,n2=np.shape(im)
B=np.random.randn(n1,n2)
sigma=10
ib=im+sigma*B
ib=np.clip(ib,0,255)
hv.Image(ib).opts(cmap='gray',width=400,height=400)


Ecrire un dashboard qui permet de visualiser rapidement l'effet d'un débruitage en ondelettes et qui renvoie les images originales, bruitées et débruitées ainsi que les PSNR associés aux images bruitées et débruitées.

In [ ]:
def Debruit(I,qmf,seednoise,sigma,Seuil):
    
    N1,N2 = np.shape(I)
    np.random.seed(seed=seednoise)
    bruit=np.random.normal(0,1,(N1,N2))
    
    Lmax=pywt.dwtn_max_level((N1,N2), pywt.Wavelet(qmf))
    
    # On bruite notre image
    IB=I+sigma*bruit
    
    IB=np.clip(IB,0,255)
    
    Seuil = Seuil*sigma 
    Irec=SeuillageDurOndelettes(IB,qmf,Lmax,Seuil)
    
    psnr1=PSNR(I,IB)
    psnr2=PSNR(I,Irec)
    
    return Irec,IB,psnr1,psnr2

In [ ]:
 #Irec, IB, p1, p2=Debruit(im,'db2',6,30,6)
# pn.Row(hv.Image(im).opts(cmap='gray',width=400,height=400),hv.Image(IB).opts(cmap='gray',width=400,height=400),hv.Image(Irec).opts(cmap='gray',width=400,height=400))

In [ ]:
def DebruitPSNR(I,qmf,N,sigma,Seuil):
    seednoise=np.arange(N)
    N1,N2  = I.size
    Lmax=pywt.dwtn_max_level((N1,N2), pywt.Wavelet(qmf))
    Seuil = Seuil*sigma
    psnr1=np.zeros(N)
    for k in seednoise:
        np.random.seed(seed=seednoise)
        bruit=np.random.normal(0,1,(N1,N2))
        IB=I+sigma*bruit
        Irec=SeuillageDurOndelettes(IB,qmf,Lmax,Seuil)
        psnr1[k]=PSNR(I,Irec)
    return np.mean(psnr1)

In [ ]:
class ImageDebruit(param.Parameterized):
    image = param.ObjectSelector(default="Lenna",objects=imagesRef.keys())
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    L = param.Integer(5,bounds=(0,7))
    Seuil = param.Number(3,bounds=(1,6))
    Sigma = param.Number(10,bounds=(1,30))
    seednoise = param.Integer(1,bounds=(0,50))
    def view(self):
        I=imagesRef[self.image]
        Irec,IB,p1,p2=Debruit(I,self.wave,self.seednoise,self.Sigma,self.Seuil)
        im=hv.Image(I).opts(cmap='gray',width=400,height=400)
        imB=hv.Image(IB).opts(title = f'Image bruitée, PSNR = {p1}',cmap='gray',width=400,height=400)
        imRec=hv.Image(Irec).opts(title = f'Image reconstruite, PSNR = {p2}',cmap='gray',width=400,height=400)
        return pn.Row( pn.Column(im), pn.Column(imB),pn.Column(imRec))

In [ ]:
imdebruit = ImageDebruit()
#pn.Row(imdebruit.param,imdebruit.view)

## Débruitage d'images et translations

Ecrire une fonction qui réalise un débruitage avec une moyenne sur des NbT fois NbT translations et la tester. Vérifier le gain en PNSR.

In [ ]:
def DebruitTranslation(IB,wave,seednoise,seuil,NbT):
    N1,N2 = I.shape
    np.random.seed(seed=seednoise)
    bruit=np.random.normal(0,1,(N1,N2))
    Lmax=pywt.dwtn_max_level((N1,N2),pywt.Wavelet(wave))
    ISum=0*IB
    P=np.zeros(NbT)
    for dx in np.arange(0,NbT):
        for dy in np.arange(0,NbT):
            IBtemp=np.roll(np.roll(IB,dx,axis=0),dy,axis=1)
            Irectemp=SeuillageDurOndelettes(IBtemp,wave,Lmax,seuil)
            Irectemp2=np.roll(np.roll(IB,-dx,axis=0),-dy,axis=1)
            ISum+=Irectemp2
    Irec=ISum/(NbT*NbT)
    return Irec

In [ ]:
I = im
wave = 'haar'
seednoise = 10
sigma = 10
Seuil = 3
NbT = 5

Irec, IB, p1, p2 = Debruit(I,wave,seednoise,sigma,Seuil)
seuil = sigma*Seuil
Irec_trans = DebruitTranslation(IB,wave,seednoise,seuil,NbT)
p3 = PSNR(I,Irec_trans)

im_originale = hv.Image(I).opts(title = "Image originale",cmap='gray',width=400,height=400)
im_bruitee = hv.Image(IB).opts(title = f'image bruitée, PSNR = {p1}',cmap='gray',width=400,height=400)
im_rec = hv.Image(Irec_trans).opts(title = f'image reconstituée, PSNR = {p3}',cmap='gray',width=400,height=400)

#pn.Row(im_originale,im_bruitee,im_rec)

Créer un dasboard pour explorer la fonction précédente. La sortie doit aussi être composée de 3 images et 2 PSNR.

In [ ]:
class Debruit_translat(param.Parameterized):
    image = param.ObjectSelector(default="Canaletto",objects=imagesRef.keys())
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    NbT = param.Integer(2,bounds=(1,8))
    Seuil = param.Number(3,bounds=(1,6))
    Sigma = param.Number(10,bounds=(1,30))
    seednoise = param.Integer(1,bounds=(0,50))
    def view(self):
        I=imagesRef[self.image]
        Irec,IB,p1,p2=Debruit(I,self.wave,self.seednoise,self.Sigma,self.Seuil)
        seuil = self.Sigma*self.Seuil
        Irec_trans = DebruitTranslation(IB,self.wave,self.seednoise,seuil,self.NbT)
        p3 = PSNR(I,Irec_trans)
        im=hv.Image(I).opts(title = "Image Originale",cmap='gray',width=400,height=400)
        imB=hv.Image(IB).opts(title = f'Image bruitée, PSNR = {p1}',cmap='gray',width=400,height=400)
        imRec=hv.Image(Irec).opts(title = f'Image reconstruite, PSNR = {p3}',cmap='gray',width=400,height=400)
        return pn.Row( pn.Column(im), pn.Column(imB),pn.Column(imRec))

In [ ]:
imdebruit_trans = Debruit_translat()
#pn.Row(imdebruit_trans.param,imdebruit_trans.view)

## Débruitage d'une image couleur.

Pour effectuer le débruitage d'une image générale, c'est à dire d'une image couleur dont le format n'est pas carré et dont les dimensions ne sont pas des puissnaces de 2 on procède comme suit :

1) On effectue un débruitage séparé sur chacun des canaux.

2) Le format carré n'est pas un vraiu problème, il faut juste que les dimensions soit des multiples de puissances de 
2. C'est la puissance de 2 qui définira l'échalle maximale de la décomposition en ondelettes. Il est donc préférable que les dimensions de l'images soient un petit multiple d'une puissance de 2.

3) On étend l'image par symétrie ou périodicité pour qu'elle ait les dimensions souhaitées. A la fin du processus de débruitage on tronque le résultat obtenu à la dimension de l'image originale.

4) Si le niveau de bruit n'est pas connu, il faut l'estimer en utilisant les coefficients d'ondelettes de la plus petite échelle (voir le notebook sur le débruitage de signaux).


# Proposer une fonction qui effectue le débruitage d'une image couleur de dimensions quelconques. 

La fonction peut prendre en entrée un tableau numpy ou une image dans une format d'images classique.
Vous pouvez tester votre programme en bruitant vous même une ou plusieurs images de référence et évaluer le gain en terme de PSNR.

In [ ]:
def Debruit_CI(I, qmf, seednoise, sigma=None, Seuil=None):
    # Séparation des canaux
    N1, N2, N3 = I.shape
    denoised_image = np.zeros((N1,N2,N3))
    
    for channel in range(N3):
        Irec,IB,psnr1,psnr2 = Debruit(I[:,:,channel],qmf,seednoise,sigma,Seuil)
        denoised_image[:,:,channel]=Irec
    psnr = PSNR(I,denoised_image)
    return denoised_image, psnr    


## Pour aller plus loin (à titre informatif et optionnel)

On peut améliorer les méthodes par seuillage dans une base d'ondelettes en effectuant un seuillage par blocs. C'est à dire, ne pas décider de conserver ou pas un coefficients en fonction de sa seule amplitude mais plutôt en fonction de l'énergie d'un voisinage de coefficients. 

Voir : http://www.cnrs.fr/insmi/spip.php?article265

En effet, il est rare qu'un coefficient soit significatif seul au milieu de coefficients nuls. 

La mméthode de sueillage par blocs consiste à choisir une taille de voisinage (par exemple 4*4 coeffients en dimension 2) pour une échelle et une direction donnée et de conserver l'intégralité des coefficients si l'énergie (la somme des carrés des coefficients) est supérieure à un seuil et de les mettre tous à 0 si ce n'est pas le cas. 

Dans ce cas aussi, les translations permettent d'améliorer le rendu visuel en limitant les effets de blocs.

On peut aussi constuire des blocs "3D" en considérant des blocs qui comprennent les coefficients des 3 créneaux de couleurs. L'idée est de corréler le débruitage un peu à travers l'espace et l'espace des couleurs.

Il est possible d'effectuer un débruitage en changeant d'espace colorimétrique en passant du RGB au YUV par exemple.

# Débruiter un minotaure ?

A l'aide de tout ce qui a été fait précédemment, proposer une version débruitée de l'image couleur contenue dans le tableau Mi

In [ ]:
Mi=chargeData('Minotaure')
Minotaure=np.clip(Mi,0,255)
#hv.RGB(Minotaure.astype('uint8')).opts(xlabel=None,ylabel=None,width=400,height=500)

In [ ]:
Mi.shape

Rédiger également une fonction prenant en entrée un nom de fichier 
permettant de calculer le PSNR de votre proposition d'image débruitée avec l'image en question.
On calcule le PSNR entre deux images couleurs en calculant la somme des erreurs quadratiques sur les 3 canaux.

Attention, l'image a 3 canaux de couleur, n'est pas carrée et les dimensions ne sont pas des puissances de 2.

In [ ]:

wave = 'haar'
seednoise = 10
sigma = 10
Seuil = 3

Irec_c, p2 = Debruit_CI(Minotaure,wave,seednoise,sigma,Seuil)
min_o = hv.RGB(Minotaure.astype('uint8')).opts(xlabel=None,ylabel=None,width=400,height=500)
min_r = hv.RGB(Irec_c.astype('uint8')).opts(xlabel=None,ylabel=None,width=400,height=500)

pn.Row(min_o,min_r)

In [ ]:
class Debruit_color(param.Parameterized):
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    Seuil = param.Number(3,bounds=(1,6))
    Sigma = param.Number(10,bounds=(1,30))
    seednoise = param.Integer(1,bounds=(0,50))
    def view(self):
        Irec,p2=Debruit_CI(Minotaure,self.wave,self.seednoise,self.Sigma,self.Seuil)
        seuil = self.Sigma*self.Seuil
        im=hv.RGB(Minotaure.astype('uint8')).opts(xlabel=None,ylabel=None,width=400,height=500)
        imRec=hv.RGB(Irec.astype('uint8')).opts(title = f'Image débruitée, PSNR = {p2}',xlabel=None,ylabel=None,width=400,height=500)
        return pn.Row( pn.Column(im), pn.Column(imRec))

In [ ]:
imdebruit_color = Debruit_color()
pn.Row(imdebruit_color.param,imdebruit_color.view)

### Plan d'expériences pour évaluer l'impact des translations  

Créer un plan d'expériences pour explorer les performances de l'invariance par translation pour le débruitage. 

In [ ]:
experiences_DebruitTrans = {'NbT':np.arange(1,5),'wave':wavelist,'Sigma':np.linspace(10,30,2)}
dfexp_DebruitTrans = pd.DataFrame(list(itertools.product(*experiences_DebruitTrans.values())),columns=experiences_DebruitTrans.keys())

In [ ]:
print(dfexp_DebruitTrans)

Ecrire une fonction qui calcule le PSNR moyen sur n réalisations de bruit du débruitage d'une image avec NbT*NbT translations (qui utilise par exemple la fonction DebruitTranslation)

In [ ]:
def Debruit_Translat_PSNRMoyen(I,wave,sigma,NbT,n):
    P=np.zeros(n)
    N1,N2=np.shape(I)
    for seednoise in np.arange(0,n):
        np.random.seed(seed=seednoise)
        bruit=np.random.normal(0,1,(N1,N2))
        IB=I+sigma*bruit
        Irec=DebruitTranslation(IB,wave,seednoise,seuil,NbT)
        P[seednoise]=PSNR(I,Irec)
    return np.mean(P)

In [ ]:
def Debruit_Translat_PSNRMoyen(I, wave, sigma, NbT, n):
    P = np.zeros(n)
    N1, N2, _ = I.shape  # Obtenez les dimensions de l'image, y compris le nombre de canaux
    for seednoise in range(n):
        np.random.seed(seed=seednoise)
        bruit = np.random.normal(0, 1, (N1, N2, 3))  # Générez un bruit pour chaque canal
        IB = I + sigma * bruit
        Irec = np.zeros_like(I)
        # Débruitage séparé sur chaque canal
        for c in range(3):
            Irec[:, :, c] = DebruitTranslation(IB[:, :, c], wave, seednoise, seuil, NbT)
        # Calcul du PSNR pour l'image débruitée
        P[seednoise] = PSNR(I, Irec)
    return np.mean(P)

Ecrire la fonction qui à une ligne de la base de données précédente calcule le PSNR moyen sur 4 réalisations du bruit. Puis l'appliquer à la base de données et ajouter la colonne des PSNR calculés à la base de données.

In [ ]:
def row2DebruitTrans(row):
    N=4
    seednoise = 10
    p=Debruit_Translat_PSNRMoyen(Minotaure,row.wave,row.Sigma,row.NbT,N)
    return {'PSNR':p}

In [ ]:
result = dfexp_DebruitTrans.apply(row2DebruitTrans,axis=1)

In [ ]:
dfexp_DebruitTrans[['PSNR']] = pd.DataFrame.from_records(result.values)

In [ ]:
# dfexp_DebruitTrans

Utiliser hvplot pour visualiser les résulatst contenus dans la base de données.

In [ ]:
h = HoverTool()
dfexp_DebruitTrans.hvplot('NbT','PSNR',by='wave',kind='scatter',groupby=['Sigma'])\
.opts(width=600,tools = [h]).redim.range(PSNR=(0,40),translation=(-0.5,10.5))

# Quantification et Entropie de Shannon

In [ ]:
def ShannonEntropy(x):
    value,counts = np.unique(x, return_counts=True)
    Proba=counts/len(x)
    Ent=-np.sum(np.log2(Proba)*Proba)
    return Ent

In [ ]:
x=np.array([13,13,2,7,13,7,1,13])
print(ShannonEntropy(x))

In [ ]:
y=np.array([-2,-3,1,0,1,0,-2,-3])
print(ShannonEntropy(y))

Ecrire une fonction qui effectue la quantification de la transformée en ondelettes avec un pas "Pas". On pourra à nouveau utiliser la commande pywt.ravel_coeffs. La fonction doit renvoyer l'image calculée par quantification, le PSNR associé ainsi que le nombre d'octets estimé par la valeur de l'entropie a priori nécessaire pour coder une telle image. On considérera qu'on code séparément les coefficients d'échelle et les coefficients d'ondelettes. Tester la fonction.

In [ ]:
def QuantificationOndelettes(I,qmf,Pas):
    coeffs = pywt.wavedecn(I, qmf)
    coeff_arr, coeff_slices, coeff_shapes = pywt.ravel_coeffs(coeffs)
    quant_scale = np.round(coeff_arr[0] / Pas) * Pas
    quant_wavelets = np.round(coeff_arr[1:] / Pas) * Pas
    quant_coeffs = pywt.unravel_coeffs((quant_scale,) + tuple(quant_wavelets), coeff_slices, coeff_shapes)
    image_quant = pywt.waverecn(quant_coeffs,qmf)
    psnr = PSNR(I, image_quant)
    entropy_scale = sum(abs(np.diff(np.sort(np.unique(quant_scale)))))
    entropy_wavelets = sum(abs(np.diff(np.sort(np.unique(quant_wavelets)))))
    bytes_needed = (entropy_scale + entropy_wavelets) * np.prod(I.shape) / 8
    return image_quant, psnr, bytes_needed

Créer le dashboard asscoié à la focntion précédente. 
Le dashboard doit renvoyer l'image quantifiée, le PSNR de l'image ainsi que le facteur de compression théorique associé. 

In [ ]:
class WaveQuant(param.Parameterized):
    image = param.ObjectSelector(default="Canaletto",objects=imagesRef.keys())
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    QS = param.Number(30,bounds=(10,300))
    def view(self) :
        Iq,psnr,B = QuantificationOndelettes(imagesRef[self.image],self.wave,self.QS)
        im=hv.Image(imagesRef[self.image]).opts(title = f'Image originale', cmap='gray',width=400,height=400)
        imQ=hv.Image(Iq).opts(title = f'Image reconstruite, PSNR = {psnr}, Bytes = {B}',cmap='gray',width=400,height=400)
        return pn.Row( pn.Column(im), pn.Column(imQ))
        

In [ ]:
Wave_Quan = WaveQuant()
#pn.Row(Wave_Quan.param,Wave_Quan.view)

Créer dun plan d'expériences pour comparer les différentes ondelettes pour la quantification... et poursuivre jusqu'à obtenir un affichage de la base de données ainsi créée avec hvplot.

In [ ]:
experiences_quant = {'Image':imagesRef, 'QS':np.linspace(30,200,10),'wave':wavelist}

In [ ]:
dfexp_experiences_quant = pd.DataFrame(list(itertools.product(*experiences_quant.values())),columns=experiences_quant.keys())

In [ ]:
dfexp_experiences_quant

In [ ]:
def row2DistorsionRate(row):
    I,p,B=QuantificationOndelettes(imagesRef[row.Image],row.wave,row.QS)
    return {'PSNR':p}

In [ ]:
result = dfexp_experiences_quant.apply(row2DistorsionRate,axis=1)

In [ ]:
dfexp_experiences_quant[['PSNR']] = pd.DataFrame.from_records(result.values)

In [ ]:
dfexp_experiences_quant

## Pour aller plus loin (à titre informatif et optionnel)

Nous proposons ici d'effectuer la compression sur les 3 canau RGB. Or l'oeil humain est plus sensible à la luminance qu'aux composantes purement chromatiques. C'est pourquoi, la plupart des algorithmes de compressions sont effectué dans un espace colorimétrique YUV où Y est la luminance. On alloue alors plus d'information au canal Y et on comprime plus drastiquement les deux autres canaux. Une méthode standart consiste par exemple à sous-échantionner d'un facteur 2 les deux composantes U et V avant de les comprimer. 

https://fr.wikipedia.org/wiki/Sous-échantillonnage_de_la_chrominance

On obtient alors des images de chrominances moins résolues et donc moins lourdes mais le rendu final reste correct car l'oeil humain est nettement plus sensible à la luminance. 